<a href="https://colab.research.google.com/github/rahulkarthik279/CS6320_Assignment1_Group11/blob/main/CS_6320_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 6320 A1 Code

Upload the training and validation text files by running this cell and clicking "Choose Files" below

In [ ]:
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()

Saving train.txt to train.txt


Saving val.txt to val.txt


### 3 - Data Preprocessing and Unsmoothed n-grams

Start placing the space-delimited tokens in an array for use. In this implementation, we treated the punctuation as necessary tokens for probability calculations. In addition, we decided to add start/stop tokens to indicate the beginning and end of each sentence. We also converted each token to all lowercase.

In [ ]:
train_tokens = []
with open("train.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.lower()
        tokens = line.strip().split(" ")
        sentence = ["<s>"]
        for token in tokens:
            sentence.append(token)
            if token in {".", "!", "?"}:
                sentence.append("</s>")
                train_tokens.extend(sentence)
                sentence = ["<s>"]
        if sentence != ["<s>"]:
            sentence.append("</s>")
            train_tokens.extend(sentence)

print("List of tokens in the training set: ", train_tokens)

Parse through the training tokens list and keep count of each unique token in the unigram dictionary.

In [ ]:
dict = {}
for token in train_tokens:
    if token in dict:
        dict[token] += 1
    else:
        dict[token] = 1

total_tokens = sum(dict.values())

print("Dictionary of word to count prior to unknown word handling: ", dict)
print(total_tokens)

For unigram implementation, get the probability by dividing token count with total unique token count.

In [ ]:
uni_prob = {}
for token, count in dict.items():
    uni_prob[token] = count / total_tokens

print("Dictionary of word to probability prior to unknown word handling: ", uni_prob)
print(len(uni_prob))

Form all possible consecutive pairs between the tokens in the tokens list and add them to the bigram dictionary.

In [ ]:
bi_dict = {}
for i in range(len(train_tokens) - 1):
  pair = (train_tokens[i], train_tokens[i + 1])
  if pair in bi_dict:
    bi_dict[pair] += 1
  else:
    bi_dict[pair] = 1

print("Dictionary of bigram to count prior to unknown word handling: ", bi_dict)

For bigram implementation, using the bigram dictionary, calculate all the probabilites of the pairs by dividing the count of the pair with the count of the given word, which can be found using the original unigram dictionary.

In [ ]:
bi_prob = {}
for (word1, word2), count in bi_dict.items():
  bi_prob[(word1, word2)] = count / dict[word1]

print("Dictionary of bigram to probability prior to unknown word handling: ", bi_prob)
print(len(bi_prob))

### 4 - Unknown word handling and smoothing

We handle unknown words here using the unknown word token approach, where we replace all words in the training set with a count of 1 with unk, since the word is rare anyway.

In [ ]:
unknown_train_tokens = []
for i in range(0, len(train_tokens)):
  if dict[train_tokens[i]] <= 1:
    unknown_train_tokens.append("<unk>")
  else:
    unknown_train_tokens.append(train_tokens[i])
unknown_word_handling_dict = {}
for token in unknown_train_tokens:
    if token in unknown_word_handling_dict:
        unknown_word_handling_dict[token] += 1
    else:
        unknown_word_handling_dict[token] = 1
print("Dictionary of word to count with unknown word handling: ", unknown_word_handling_dict)

In [ ]:
unknown_word_handling_dict_prob = {}
for token, count in unknown_word_handling_dict.items():
    unknown_word_handling_dict_prob[token] = count / sum(unknown_word_handling_dict.values())

print("Dictionary of word to probability with unknown word handling: ", unknown_word_handling_dict_prob)
print(len(unknown_word_handling_dict_prob))

In [ ]:
unknown_bi_dict = {}
for i in range(len(unknown_train_tokens) - 1):
  pair = (unknown_train_tokens[i], unknown_train_tokens[i + 1])
  if pair in unknown_bi_dict:
    unknown_bi_dict[pair] += 1
  else:
    unknown_bi_dict[pair] = 1

print("Bigram dictionary with unknown token: ", unknown_bi_dict)

After getting the unigram and bigram probabilities, we can smooth the probabilities. Since we dealt with unknown words with a unk token, the unigrams don't need to be smoothed, since every word is already assigned a probability. Add-K smoothing for bigrams involves creating all possible 2-token combinations and giving them a value.

In [ ]:
# LAPLACE SMOOTHING
laplace_smoothed_unknown_bi_dict = {}
for pair, count in unknown_bi_dict.items():
  laplace_smoothed_unknown_bi_dict[pair] = (count + 1)/(unknown_word_handling_dict[pair[0]] + len(unknown_word_handling_dict.keys()))

print("Laplace smoothed dict:", laplace_smoothed_unknown_bi_dict)

# Add-K SMOOTHING (0.10, 0.25, 0.50)
K = 0.10
add_010k_smoothed_unknown_bi_dict = {}
for pair, count in unknown_bi_dict.items():
  add_010k_smoothed_unknown_bi_dict[pair] = (count + K)/(unknown_word_handling_dict[pair[0]] + len(unknown_word_handling_dict.keys()) * K)

K = 0.25
add_025k_smoothed_unknown_bi_dict = {}
for pair, count in unknown_bi_dict.items():
  add_025k_smoothed_unknown_bi_dict[pair] = (count + K)/(unknown_word_handling_dict[pair[0]] + len(unknown_word_handling_dict.keys()) * K)

K = 0.50
add_050k_smoothed_unknown_bi_dict = {}
for pair, count in unknown_bi_dict.items():
  add_050k_smoothed_unknown_bi_dict[pair] = (count + K)/(unknown_word_handling_dict[pair[0]] + len(unknown_word_handling_dict.keys()) * K)

print("Add-k smoothed dict with k = 0.1: ", add_010k_smoothed_unknown_bi_dict)
print("Add-k smoothed dict with k = 0.25: ", add_025k_smoothed_unknown_bi_dict)
print("Add-k smoothed dict with k = 0.5: ", add_050k_smoothed_unknown_bi_dict)

In [ ]:
val_tokens = []
with open("val.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.lower()
        tokens = line.strip().split(" ")
        sentence = ["<s>"]
        for token in tokens:
            sentence.append(token)
            if token in {".", "!", "?"}:
                sentence.append("</s>")
                val_tokens.extend(sentence)
                sentence = ["<s>"]
        if sentence != ["<s>"]:
            sentence.append("</s>")
            val_tokens.extend(sentence)

for i in range(0, len(val_tokens)):
  if val_tokens[i] not in list(unknown_word_handling_dict.keys()):
    val_tokens[i] = "<unk>"
print("List of tokens in the validation set: ", val_tokens)

### 5 - Perplexity

Unigram Perplexity Calculation

In [ ]:
import math
perp_sum = 0
for token in val_tokens:
  prob = unknown_word_handling_dict_prob[token]
  perp_sum += math.log(prob) * -1

unigram_perplexity = math.exp(perp_sum / len(val_tokens))
print("Unigram - perplexity: ", unigram_perplexity)

Unigram - perplexity:  239.7254702907334


Bigram Laplace Perplexity

In [ ]:
import math
perp_sum = 0
for i in range(0, len(val_tokens)-1):
  pair = (val_tokens[i], val_tokens[i+1])
  if pair != ("</s>", "<s>"):
    try:
      prob = laplace_smoothed_unknown_bi_dict[pair]
    except KeyError:
      prob = 1/(unknown_word_handling_dict[pair[0]] + len(unknown_word_handling_dict.keys()))
    perp_sum += math.log(prob) * -1

bigram_laplace_perplexity = math.exp(perp_sum / len(val_tokens))
print("Laplace smoothing for bigram - perplexity: ", bigram_laplace_perplexity)

Laplace smoothing for bigram - perplexity:  233.99399201670414


Bigram Add K Perplexity

In [ ]:
import math
sum_add01 = 0
sum_add025 = 0
sum_add05 = 0
for i in range(0, len(val_tokens)-1):
  pair = (val_tokens[i], val_tokens[i+1])
  if pair != ("</s>", "<s>"):
    try:
      prob01 = add_010k_smoothed_unknown_bi_dict[pair]
    except KeyError:
      prob01 = 1/(unknown_word_handling_dict[pair[0]] + 0.1 * len(unknown_word_handling_dict.keys()))
    try:
      prob025 = add_025k_smoothed_unknown_bi_dict[pair]
    except KeyError:
      prob025 = 1/(unknown_word_handling_dict[pair[0]] + 0.25 * len(unknown_word_handling_dict.keys()))
    try:
      prob05 = add_050k_smoothed_unknown_bi_dict[pair]
    except KeyError:
      prob05 = 1/(unknown_word_handling_dict[pair[0]] + 0.5 * len(unknown_word_handling_dict.keys()))
    sum_add01 += math.log(prob01) * -1
    sum_add025 += math.log(prob025) * -1
    sum_add05 += math.log(prob05) * -1

bigram_add_01_perplexity = math.exp(sum_add01 / len(val_tokens))

bigram_add_025_perplexity = math.exp(sum_add025 / len(val_tokens))

bigram_add_05_perplexity = math.exp(sum_add05 / len(val_tokens))

print("Add k smoothing for bigram with k = 0.1 - perplexity: ", bigram_add_01_perplexity)
print("Add k smoothing for bigram with k = 0.25 - perplexity: ", bigram_add_025_perplexity)
print("Add k smoothing for bigram with k = 0.5 - perplexity: ", bigram_add_05_perplexity)

Add k smoothing for bigram with k = 0.1 - perplexity:  69.24696162663253
Add k smoothing for bigram with k = 0.25 - perplexity:  105.93249718837934
Add k smoothing for bigram with k = 0.5 - perplexity:  154.40809162430068
